In [1]:
# Fetch Iris Dataset from https://archive.ics.uci.edu/ml/datasets/Iris
# Make attached Python script, softmax_irises.py work. 
# You might have to upgrade the script to TF 1.x API.  

In [ ]:
# https://www.tensorflow.org/install/migration
# https://github.com/tensorflow/tensorflow/issues/11217
# python2.7 tf_upgrade.py --infile softmax_irises-1.py  --outfile softmax_irises-2.py

- tensorboard --logdir 'problem-3'

In [2]:
# Softmax example in TF using the classical Iris dataset
# Download iris.data from https://archive.ics.uci.edu/ml/datasets/Iris

import tensorflow as tf
import os

In [3]:
# weights form a matrix, not a column vector, one "weight vector" per class.
W = tf.Variable(tf.zeros([4, 3]), name="weights")
# so do the biases, one per class.
b = tf.Variable(tf.zeros([3], name="bias"))

In [4]:
def combine_inputs(X):
    return tf.matmul(X, W) + b

In [5]:
def inference(X):
    return tf.nn.softmax(combine_inputs(X))

In [6]:
def loss(X, Y):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=combine_inputs(X), labels=Y))

In [7]:
def read_csv(batch_size, file_name, record_defaults):
    # filename_queue = tf.train.string_input_producer([os.path.dirname(__file__) + "/" + file_name])
    filename_queue = tf.train.string_input_producer([file_name])
    print file_name
    print filename_queue


    reader = tf.TextLineReader(skip_header_lines=1)
    print reader
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)

In [8]:
def inputs():
    sepal_length, sepal_width, petal_length, petal_width, label =\
        read_csv(100, "iris.data", [[0.0], [0.0], [0.0], [0.0], [""]])

    # convert class names to a 0 based class index.
    label_number = tf.to_int32(tf.argmax(tf.to_int32(tf.stack([
        tf.equal(label, ["Iris-setosa"]),
        tf.equal(label, ["Iris-versicolor"]),
        tf.equal(label, ["Iris-virginica"])
    ])), 0))

    # Pack all the features that we care about in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    features = tf.transpose(tf.stack([sepal_length, sepal_width, petal_length, petal_width]))

    return features, label_number

In [9]:
def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [10]:
def evaluate(sess, X, Y):

    predicted = tf.cast(tf.arg_max(inference(X), 1), tf.int32)

    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))

In [14]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:

    tf.global_variables_initializer().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    writer = tf.summary.FileWriter("problem-3", graph=tf.get_default_graph())
    tf.summary.scalar('loss1', total_loss)

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            writer.add_summary(sess.run(tf.summary.merge_all()), global_step=step)
            print("Step {}, loss: ".format(step), sess.run([total_loss]))

    evaluate(sess, X, Y)

    coord.request_stop()
    coord.join(threads)
    sess.close()

iris.data
('Step 0, loss: ', [1.0898832])


('Step 10, loss: ', [1.0148832])


('Step 20, loss: ', [0.99335462])


('Step 30, loss: ', [0.91635209])


('Step 40, loss: ', [0.86754453])


('Step 50, loss: ', [0.8247748])


('Step 60, loss: ', [0.83856642])


('Step 70, loss: ', [0.79748917])


('Step 80, loss: ', [0.79458284])


('Step 90, loss: ', [0.78015739])


('Step 100, loss: ', [0.71979094])


('Step 110, loss: ', [0.70995116])


('Step 120, loss: ', [0.69566315])


('Step 130, loss: ', [0.65042061])


('Step 140, loss: ', [0.6102156])


('Step 150, loss: ', [0.59877431])


('Step 160, loss: ', [0.58082783])


('Step 170, loss: ', [0.62390381])


('Step 180, loss: ', [0.58097643])


('Step 190, loss: ', [0.58120489])


('Step 200, loss: ', [0.56746191])


('Step 210, loss: ', [0.55816239])


('Step 220, loss: ', [0.56133652])


('Step 230, loss: ', [0.53153962])


('Step 240, loss: ', [0.57969779])


('Step 250, loss: ', [0.58183497])


('Step 260, loss: ', [0.5585283])


('Step 